In [1]:
import sys
sys.path.append('../')
import exp
import regression as r

In [2]:
df = exp.get_wp_energy_data_8_day()
df.head()

,PET,VPD,air_temp,doy,precip,soil_temp,sw_in,wind_speed,year,wp_RNET,...,wp_bnd3,wp_bnd7,wp_evi,wp_lswi2,wp_ndvi,wp_LST.day,wp_LST.night,LW_IN.wp,LW_IN.si,LW_IN
0,7.33,0.808731,19.179167,195,0.0,22.320833,30.3156,4.958333,2012,20.798342,...,0.032962,0.136763,0.396594,0.353988,0.641934,31.567899,17.204530,29.105844,24.562860,29.105844
1,6.52,0.755945,19.325000,196,0.0,21.770833,29.6316,3.791667,2012,20.573593,...,0.032200,0.131800,0.407211,0.376832,0.654545,29.570000,17.390000,29.760736,26.096551,29.760736
2,6.92,0.858993,20.262500,197,0.0,21.908333,29.3472,4.137500,2012,20.475931,...,0.031425,0.128250,0.416158,0.394189,0.664269,31.097908,17.235624,30.299442,25.986418,30.299442
3,6.35,0.477617,16.791667,198,0.0,22.420833,28.8180,6.033333,2012,20.571045,...,0.030650,0.124700,0.425106,0.411547,0.673993,30.868718,17.248525,28.978868,25.122209,28.978868
4,5.13,0.556820,17.016667,199,0.0,21.529167,23.1732,4.350000,2012,16.757401,...,0.029875,0.121150,0.434053,0.428904,0.683716,30.657792,17.259663,30.653667,27.830367,30.653667


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1030 entries, 0 to 1029
Data columns (total 28 columns):
PET             1030 non-null float64
VPD             1030 non-null float64
air_temp        1030 non-null float64
doy             1030 non-null int64
precip          1030 non-null float64
soil_temp       1030 non-null float64
sw_in           1030 non-null float64
wind_speed      1030 non-null float64
year            1030 non-null int64
wp_RNET         1030 non-null float64
wp_ch4_gf       1030 non-null float64
wp_co2_gf       1030 non-null float64
wp_er           1030 non-null float64
wp_gpp          1030 non-null float64
wp_h            1030 non-null float64
wp_le           1030 non-null float64
wp_bnd1         1030 non-null float64
wp_bnd2         1030 non-null float64
wp_bnd3         1030 non-null float64
wp_bnd7         1030 non-null float64
wp_evi          1030 non-null float64
wp_lswi2        1030 non-null float64
wp_ndvi         1030 non-null float64
wp_LST.day      1030 no

In [4]:
train_cols = ["precip", "sw_in", "LW_IN.wp", "VPD", "air_temp", "PET", "wind_speed", "wp_LST.day",
             "wp_LST.night", "wp_evi", "wp_ndvi", "wp_lswi2"]
X, Y = exp.featurize(df, train_cols, ["wp_gpp"])
X, Y, scaler = r.preprocess(X, Y)
X.shape

(1030, 12)

In [5]:
r.random_forests_cross_val(X, Y, feature_names=train_cols)

Running Random Forests Cross Validation...
10-fold CV Acc Mean:  0.976298573805
CV Scores:  0.983980569247, 0.976848228394, 0.975041902955, 0.976051526692, 0.979015228842, 0.979439658373, 0.966017997086, 0.976139234758, 0.980050563388, 0.970400828317
OOB score: 0.977155575437
Feature Importances:
('wp_LST.night', 0.30423218672623376)
('air_temp', 0.19060939719765163)
('wp_LST.day', 0.14650224559044994)
('PET', 0.094189026420581107)
('sw_in', 0.05664226226575713)
('VPD', 0.053707174788686453)
('LW_IN.wp', 0.04511500403994613)
('wp_evi', 0.044516321763418139)
('wp_ndvi', 0.028261988806870475)
('wp_lswi2', 0.024612223684124995)
('wind_speed', 0.010097111706209656)
('precip', 0.0015150570100705457)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='sqrt', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=200, n_jobs=1, oob_score=True, random_state=None,
           verbose=0, warm_start=False)

In [6]:
r.xgb_trees_cross_val(X, Y, feature_names=train_cols)

Running Gradient Boosted Trees Cross Validation...
10-fold CV Acc Mean:  0.974484577702
CV Scores:  0.987250822283, 0.974524172317, 0.975500426108, 0.982190450787, 0.972524441373, 0.969287235221, 0.961229878077, 0.971599235073, 0.980149133477, 0.970589982303
Feature Importances:
('wp_LST.night', 0.17368933851044893)
('wp_evi', 0.16092089795902528)
('wp_LST.day', 0.11855407906282255)
('wp_lswi2', 0.10646571755631942)
('wp_ndvi', 0.07721132441622866)
('air_temp', 0.066428552485865253)
('LW_IN.wp', 0.063669743892705868)
('VPD', 0.062389477880498444)
('sw_in', 0.060129902855892627)
('wind_speed', 0.053827836534783839)
('PET', 0.049797498110022681)
('precip', 0.0069156307353865432)


GradientBoostingRegressor(alpha=0.9, init=None, learning_rate=0.1, loss='ls',
             max_depth=3, max_features='sqrt', max_leaf_nodes=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=200,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False)

In [7]:
r.svc_cross_val(X, Y)

Running SVC Cross Validation...
10-fold CV Acc Mean:  0.836637092578
CV Scores:  0.877525613692, 0.858870701148, 0.803344667378, 0.813846820106, 0.848901487975, 0.844427546556, 0.804636358155, 0.808175702082, 0.866880185926, 0.839761842763


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [8]:
r.dnn_cross_val(X, Y)

Running Neural Network Cross Validation...
Step #1, avg. loss: 2634.96875
Step #501, epoch #50, avg. loss: 128.37437
Step #1001, epoch #100, avg. loss: 17.23398
Step #1501, epoch #150, avg. loss: 10.85658
Step #2001, epoch #200, avg. loss: 8.14879
Step #2501, epoch #250, avg. loss: 6.65521
Step #3001, epoch #300, avg. loss: 5.55305
Step #3501, epoch #350, avg. loss: 4.76765
Step #4001, epoch #400, avg. loss: 4.16136
Step #4501, epoch #450, avg. loss: 3.68826
Step #1, avg. loss: 2708.00903
Step #501, epoch #50, avg. loss: 129.72310
Step #1001, epoch #100, avg. loss: 16.97992
Step #1501, epoch #150, avg. loss: 10.57328
Step #2001, epoch #200, avg. loss: 7.79752
Step #2501, epoch #250, avg. loss: 6.20455
Step #3001, epoch #300, avg. loss: 5.19628
Step #3501, epoch #350, avg. loss: 4.38987
Step #4001, epoch #400, avg. loss: 3.95249
Step #4501, epoch #450, avg. loss: 3.56524
Step #1, avg. loss: 2595.48438
Step #501, epoch #50, avg. loss: 127.43584
Step #1001, epoch #100, avg. loss: 15.89498

TensorFlowEstimator(batch_size=100, class_weight=None,
          continue_training=False, early_stopping_rounds=None,
          keep_checkpoint_every_n_hours=10000, learning_rate=0.1,
          max_to_keep=5, model_fn=<function tanh_dnn at 0x11873fb18>,
          n_classes=0, num_cores=4, optimizer='SGD', steps=5000,
          tf_master='', tf_random_seed=42, verbose=1)